In [51]:
from mlflow.tracking.client import MlflowClient
from kubernetes import client, config

IP_MLFLOW = "http://34.65.82.22"
MODEL_NAME = "iris_logistic_regression"

mlflow_client = MlflowClient(tracking_uri=IP_MLFLOW)
model = mlflow_client.get_latest_versions(MODEL_NAME)[0]
model_version = model.version
model_uri = mlflow_client.get_model_version_download_uri(
    MODEL_NAME, model_version
)

/tmp/ipykernel_1112840/977019740.py:8: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  model = mlflow_client.get_latest_versions(MODEL_NAME)[0]


In [52]:
print(f"Latest model URI: {model_uri}")

Latest model URI: mlflow-artifacts:/7/e0b43e2b3f144023819d58438b80f002/artifacts/model


In [56]:
K8S_NAMESPACE = "mlflow-kserve"
K8S_DEPLOYMENT_NAME = "iris-logistic-regression-inference"


apps_v1 = client.AppsV1Api()

# Définir ou mettre à jour le déploiement Kubernetes
deployment = {
    "apiVersion": "apps/v1",
    "kind": "Deployment",
    "metadata": {"name": K8S_DEPLOYMENT_NAME, "namespace": K8S_NAMESPACE},
    "spec": {
        "replicas": 3,  # Nombre de pods pour servir l'inférence
        "selector": {"matchLabels": {"app": K8S_DEPLOYMENT_NAME}},
        "template": {
            "metadata": {"labels": {"app": K8S_DEPLOYMENT_NAME}},
            "spec": {
                "containers": [
                    {
                        "name": "inference",
                        "image": "ghcr.io/seldonio/mlserver:latest",  # Utilise MLServer comme serveur
                        "args": [
                            "--model-uri",
                            model_uri,
                            "--protocol",
                            "v2",
                        ],
                        "ports": [{"containerPort": 8080}],
                    }
                ]
            },
        },
    },
}


In [ ]:

# Appliquer ou mettre à jour le déploiement
try:
    existing_deployment = apps_v1.read_namespaced_deployment(
        name=K8S_DEPLOYMENT_NAME, namespace=K8S_NAMESPACE
    )
    apps_v1.patch_namespaced_deployment(
        name=K8S_DEPLOYMENT_NAME, namespace=K8S_NAMESPACE, body=deployment
    )
    print(f"Deployment {K8S_DEPLOYMENT_NAME} updated successfully.")
except client.exceptions.ApiException as e:
    if e.status == 404:
        apps_v1.create_namespaced_deployment(
            namespace=K8S_NAMESPACE, body=deployment
        )
        print(f"Deployment {K8S_DEPLOYMENT_NAME} created successfully.")
    else:
        raise
